In [7]:
import os

import numpy as np

import torch
import torch.nn as nn
import torchvision

from simclr import SimClrBackbone

from util import *

In [8]:
device = "cuda"

In [9]:
from torch.utils.data import DataLoader
from torchvision import transforms

transform_train= transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


batch_size = 1024
train_dataset = torchvision.datasets.CIFAR10(root='C:/Datasets', train=True, download=False, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='C:/Datasets', train=False, download=False, transform=transform_test)
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [10]:
class SimClrLR(nn.Module):
    def __init__(self):
        super().__init__()
        
        dataset_name = "clean"
        simclr_model_name = f"{dataset_name}-SimCLR.pt"
        self.simclr, _ = load_simclr(simclr_model_name)
        
        for p in self.simclr.parameters():
            p.requires_grad = False
            
        for p in self.simclr.projector.parameters():
            p.requires_grad = False
        
        self.lastlayer = nn.Linear(512, 10)

    def forward(self,x):
        out = self.simclr.pretrained(x)
        out = self.lastlayer(out)
        return out
    
model = SimClrLR().to(device)

In [11]:
optimizer = torch.optim.SGD([params for params in model.parameters() if params.requires_grad],lr = 0.6, momentum = 0.9, weight_decay=0., nesterov=True)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
loss_fn = nn.CrossEntropyLoss()

In [12]:
tr_ep_loss = []
tr_ep_acc = []

val_ep_loss = []
val_ep_acc = []

min_val_loss = 100.0

EPOCHS = 10
num_cl = 10

In [13]:
import time

for epoch in range(20):
    
    stime = time.time()
    print("=============== Epoch : %3d ==============="%(epoch+1))
    
    loss_sublist = np.array([])
    acc_sublist = np.array([])
    
    #iter_num = 0
    model.train()
    
    optimizer.zero_grad()
    
    for x,y in trainloader:
        x = x.squeeze().to(device, dtype = torch.float)
        y = y.to(device)
        
        z = model(x)
        
        optimizer.zero_grad()
        
        tr_loss = loss_fn(z,y)
        tr_loss.backward()

        preds = torch.exp(z.cpu().data)/torch.sum(torch.exp(z.cpu().data))
        
        optimizer.step()
        
        loss_sublist = np.append(loss_sublist, tr_loss.cpu().data)
        acc_sublist = np.append(acc_sublist,np.array(np.argmax(preds,axis=1)==y.cpu().data.view(-1)).astype('int'),axis=0)
        
    print('ESTIMATING TRAINING METRICS.............')
    
    print('TRAINING BINARY CROSSENTROPY LOSS: ',np.mean(loss_sublist))
    print('TRAINING BINARY ACCURACY: ',np.mean(acc_sublist))
    
    tr_ep_loss.append(np.mean(loss_sublist))
    tr_ep_acc.append(np.mean(acc_sublist))
    
    print('ESTIMATING VALIDATION METRICS.............')
    
    model.eval()
    
    loss_sublist = np.array([])
    acc_sublist = np.array([])
    
    with torch.no_grad():
        for x,y in testloader:
            x = x.squeeze().to(device = 'cuda:0', dtype = torch.float)
            y = y.to(device = 'cuda:0')
            z = model(x)

            val_loss = loss_fn(z,y)

            preds = torch.exp(z.cpu().data)/torch.sum(torch.exp(z.cpu().data))

            loss_sublist = np.append(loss_sublist, val_loss.cpu().data)
            acc_sublist = np.append(acc_sublist,np.array(np.argmax(preds,axis=1)==y.cpu().data.view(-1)).astype('int'),axis=0)
    
    print('VALIDATION BINARY CROSSENTROPY LOSS: ',np.mean(loss_sublist))
    print('VALIDATION BINARY ACCURACY: ',np.mean(acc_sublist))
    
    val_ep_loss.append(np.mean(loss_sublist))
    val_ep_acc.append(np.mean(acc_sublist))
    
    lr_scheduler.step()
    
    print("Time Taken : %.2f minutes"%((time.time()-stime)/60.0))

=============== Epoch :   1 ===============
ESTIMATING TRAINING METRICS.............
TRAINING BINARY CROSSENTROPY LOSS:  0.6380205866025419
TRAINING BINARY ACCURACY:  0.79048
ESTIMATING VALIDATION METRICS.............
VALIDATION BINARY CROSSENTROPY LOSS:  0.5320795387029648
VALIDATION BINARY ACCURACY:  0.819
Time Taken : 1.31 minutes
=============== Epoch :   2 ===============
ESTIMATING TRAINING METRICS.............
TRAINING BINARY CROSSENTROPY LOSS:  0.510621860927465
TRAINING BINARY ACCURACY:  0.8255
ESTIMATING VALIDATION METRICS.............
VALIDATION BINARY CROSSENTROPY LOSS:  0.5079841464757919
VALIDATION BINARY ACCURACY:  0.8242
Time Taken : 1.22 minutes
=============== Epoch :   3 ===============
ESTIMATING TRAINING METRICS.............
TRAINING BINARY CROSSENTROPY LOSS:  0.4944273859870677
TRAINING BINARY ACCURACY:  0.8291
ESTIMATING VALIDATION METRICS.............
VALIDATION BINARY CROSSENTROPY LOSS:  0.48782203197479246
VALIDATION BINARY ACCURACY:  0.8323
Time Taken : 1.23 

In [14]:
print(max(val_ep_acc))

0.8408


: 